In [1]:
from com.ibm.watson.utils import excel_utils
from com.ibm.watson.datasource import nci_graph
from com.ibm.watson import DOCS_DIR, DESKTOP
import os, pymongo

In [2]:
client = pymongo.MongoClient()
db = client["WATSON"]

In [3]:
primary_diseases = dict()
secondary_diseases = dict()
for i, row in enumerate(excel_utils.iter_xls(os.path.join(DESKTOP, "Disease_Ontology_v28(1).xlsx"), sheet_num = 1)):
    if i > 0:
        primary_disease_code = row[1].value.strip()
        secondary_disease_code = row[3].value.strip()
        primary_diseases[primary_disease_code] = primary_diseases.get(primary_disease_code, []) + [secondary_disease_code]
        secondary_diseases[secondary_disease_code] = primary_disease_code
        
print len(primary_diseases)
print len(secondary_diseases)

72
398


/Users/singhv/Library/Python/2.7/lib/python/site-packages/openpyxl/workbook/names/named_range.py:125: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


In [11]:
#with open(os.path.join(DESKTOP, "unmatched_codes.tsv"), "wb") as output_file:
for i, study in enumerate(db["CTGOV"].find({"metadata.STATUS" : "ACCEPTED"})):
    if (i% 500 == 0):
        print "Processing (%d): %s" % (i, study["NCT_ID"])
    condition_umls_codes = dict([ (condition["ID"], condition) for condition in study["conditions"] if "ID" in condition ])
    condition_nci_codes = [ (x["CODE"], condition_umls_codes[x["NCIM_CUI"]]) for x in db["NCIT"].find({"NCIM_CUI" : {"$in" : condition_umls_codes.keys()}}) ]
    #unmatched_secondary_disease_codes = set([ code for code in condition_nci_codes.keys() if (code not in secondary_diseases) ])

    for (nci_code, condition) in condition_nci_codes:
        if (nci_code in secondary_diseases) or (nci_code in primary_diseases):
            condition["CNM"] = nci_code
        else:
            parent_codes_and_distances = nci_graph.find_parents(nci_code)
            for (parent_code, distance) in parent_codes_and_distances:
                if (parent_code in secondary_diseases) or (parent_code in primary_diseases):
                    condition["CNM"] = parent_code
                    break
            
    conditions = study["conditions"]
    db["CTGOV"].update_one({"NCT_ID" : study["NCT_ID"]}, {"$set" : {"conditions" : conditions}})
        
        #unmatched_primary_disease_codes = [ code for code in unmatched_secondary_disease_codes if (code not in primary_diseases) ]
        #output_file.write("\t".join( [study["NCT_ID"]] + unmatched_primary_disease_codes + ["\n"] ))
        #print study

Processing (0): NCT01641939


KeyboardInterrupt: 